# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [3]:
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


## Разбейте данные на выборки

Мы умеем делить только на две части , поэтому сначала отделим 40% от всего датасета, а потом эти 40 % поделим пополам между тестовой и валидационной выборкой. Таким образом в них будет по 20 % от всего датасета.

In [4]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Рекомендую явно вывести размеры получившихся выборок (с использованием .shape и print), это поможет быстро оценить корректность разделения на выборки.
</div>

Подготовим поделенные данные ( поделим на прикнаки и целевой признак)

In [5]:
# Для обучения модели 
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# Для проверки модели 
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# Для оценки качества модели
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Исследуйте модели

У нас целевой прризнак это столбец "is_ultra". В нем всего два вариатна значений 1 или 0, следовательно этот признак явяется категориальным. Будем решать задачу классификации, используя модели машинного обучения: Решающее дерево, Случайный лес, Логистическая регрессия.

Решающее дерево

In [6]:
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)# обучите модель с заданной глубиной дерева
    model.fit(features_train,target_train) # обучите модель
    predictions_valid = model.predict(features_valid) # получите предсказания модели
    result = accuracy_score(target_valid, predictions_valid) # посчитайте качество модели
    print('max_depth =', depth, ':', result)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633


Лучший результат при параметре глубины: max_depth = 3 : 0.7853810264385692.

Случайный лес

In [7]:
best_model = None
best_result = 0
for depth in range(2, 11):
    for est in range(1, 21):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train,target_train) # обучите модель на тренировочной выборке
        result = model.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
        if result > best_result:
            best_model = model # сохраните наилучшую модель
            best_result = result #  сохраните наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print(best_model)

Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936
RandomForestClassifier(max_depth=6, n_estimators=12, random_state=12345)


Логистическая регрессия

In [8]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train,target_train)

predictions = model.predict(features_valid)

accuracy_score(target_valid, predictions_valid)

0.7744945567651633

Вывод: 
Сравнение моделей и их точность:
1) Решающее дерево 0.7853810264385692

2) Случайный лес 0.8040435458786936

3) Логистическая регрессия 0.7744945567651633

В итоге мы выбираем Случайный лес

## Проверьте модель на тестовой выборке

In [9]:
model = best_model
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.7947122861586314

## (бонус) Проверьте модели на адекватность

Модель уже дает 79% правильных ответов, а эт обольше чем при случайном (50%)

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
В основе проверки на адекватность лежит очень простая идея. Модель считается адекватной, если способна "побить" любой "наивный" алгоритм, например предсказывающий случайный или постояный ответ. 

Для нашего проекта самую высокую точность дает  "наивный" алгоритм, всегда предсказывающий самый популярный ответ (у нас это 0). Проверить точность такого подхода можно вручную или используя DummyClassifier.
</div>

## Вывод

Мы рассмотрели три модели: "Решающее дерево", "Случайный лес" и "Логистическая регрессия". В каждой модели, изменяя ее гиперпараметры, мы нашли нашли лучший вариант с наибольшим количеством правильных ответов. Путем простого сравнения лучших моделей по доле правильных ответов определили, что для нашей задачи подходит модель "Случайного леса" с гиперпараметрами глубина дерева 6 и количество деревьев 12.

Сравнив нашу модель со случайными предсказаниями и оценив ее качество на тестовом наборе данных (результат составил 79% правильных ответов) мы подтвердили, что наша модель подходит для задачи заказчика. Таким образом можно использовать данную модель для предложения клиентам заказчика нового тарифа ("Смарт" или "Ультра").